In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.optimize import curve_fit
from datetime import timedelta,date,datetime

In [2]:
def NormalFecha(fecha):
    try:
        '''
        if len(fecha) == 10: # Por si hay fechas tipo 2020/07/06
            fecha = fecha.replace('/','-')
            fecha=fecha.split('-')
            fecha = fecha[2] + '-' + fecha[1]+'-'+ fecha[0]
        else:
        '''
            # Haga el split adecuado
        fecha = fecha.split('T')[0].split('-')
        if len(fecha) == 3:
            fecha = fecha[0] + '-' + fecha[1]+'-'+ fecha[2]
        else:
            fecha = 'NaN'
        # Cambiaron 'SIN DATO' a una fecha       
        #if fecha=='1899-12-31': # 2020-07-06
        if fecha=='SIN DATO':
            fecha = 'NaN'
            
    except IndexError:
        fecha = 'NaN'
    return fecha

In [3]:
def Fechas_Neg(fecha):
    if len(fecha)>=11:
        fecha=str(fecha).split('\n')[0].split('days')[0]
        fecha=fecha.split('+')[0]
        fecha=float(fecha)
    return fecha


In [4]:

# Leer Datos
fecha_pura=date.today()+timedelta(days=-1)
fecha = str(fecha_pura)
print("Fecha Boletín:",fecha)

Fecha Boletín: 2020-09-11


In [8]:
datos=pd.read_csv('Casos_positivos_de_COVID-19_en_Colombia.csv',low_memory=False)

In [9]:
datos_todo=len(datos)
print("Observaciones Totales:", datos_todo,'\n')
print(datos.columns.values)

Observaciones Totales: 702088 

['ID de caso' 'Fecha de notificación' 'Código DIVIPOLA'
 'Ciudad de ubicación' 'Departamento o Distrito ' 'atención' 'Edad' 'Sexo'
 'Tipo' 'Estado' 'País de procedencia' 'FIS' 'Fecha de muerte'
 'Fecha diagnostico' 'Fecha recuperado' 'fecha reporte web'
 'Tipo recuperación' 'Codigo departamento' 'Codigo pais'
 'Pertenencia etnica' 'Nombre grupo etnico']


In [10]:
# Hacer Nombres Eficientes (En tu cara Colombia)

renombrar_columns= ['ID_caso', 'Fecha_notificacion', 'DIVIPOLA',
 'Ciudad', 'Departamento_Distrito', 'Atencion', 'Edad', 'Sexo',
 'Tipo_caso', 'Estado', 'Pais_procedencia', 'Fecha_inicio_sintoma', 'Fecha_muerte',
 'Fecha_diagnostico', 'Fecha_recuperado', 'Fecha_web',
 'Tipo_recuperacion', 'Codigo_departamento', 'Codigo_pais',
 'Pertenencia_etnica','Nombre_etnia'] # Nombre_etnia Agregado 2020-06-24

datos.columns=renombrar_columns

# Re-ordenar Nombres Para Facilitar lectura por inspección

reordenar= ['ID_caso', 'Fecha_inicio_sintoma', 'Fecha_notificacion', 'Fecha_diagnostico','Fecha_recuperado','Fecha_muerte','Fecha_web',
             'Estado','Atencion','Tipo_recuperacion','Ciudad','DIVIPOLA','Departamento_Distrito','Codigo_departamento', 'Edad', 'Sexo','Tipo_caso', 
            'Pais_procedencia', 'Codigo_pais','Pertenencia_etnica','Nombre_etnia']

# Re-ordenar Columnas

datos = datos[reordenar]

In [11]:
print("Variables Colombia:\n\n",datos.columns.values,'\n')
print("Atención:\n\n",datos["Atencion"].unique(),'\n')
print("Estado:\n\n",datos["Estado"].unique(),'\n')

Variables Colombia:

 ['ID_caso' 'Fecha_inicio_sintoma' 'Fecha_notificacion' 'Fecha_diagnostico'
 'Fecha_recuperado' 'Fecha_muerte' 'Fecha_web' 'Estado' 'Atencion'
 'Tipo_recuperacion' 'Ciudad' 'DIVIPOLA' 'Departamento_Distrito'
 'Codigo_departamento' 'Edad' 'Sexo' 'Tipo_caso' 'Pais_procedencia'
 'Codigo_pais' 'Pertenencia_etnica' 'Nombre_etnia'] 

Atención:

 ['Recuperado' 'Fallecido' nan 'Hospital UCI' 'Casa' 'Hospital'] 

Estado:

 ['Leve' 'Asintomático' 'Fallecido' 'Moderado' nan 'Grave'] 



In [12]:
#Convertir  Fechas a strings
datos['Fecha_inicio_sintoma']=datos['Fecha_inicio_sintoma'].astype('str')
datos['Fecha_notificacion']=datos['Fecha_notificacion'].astype('str')
datos['Fecha_diagnostico']=datos['Fecha_diagnostico'].astype('str')
datos['Fecha_muerte']=datos['Fecha_muerte'].astype('str')
datos['Fecha_recuperado']=datos['Fecha_recuperado'].astype('str')
datos['Fecha_muerte']=datos['Fecha_muerte'].astype('str')
datos['Fecha_web']=datos['Fecha_web'].astype('str')

#Limpiar Fechas
datos['Fecha_inicio_sintoma']=datos['Fecha_inicio_sintoma'].apply(lambda fecha: NormalFecha(fecha))
datos['Fecha_notificacion']=datos['Fecha_notificacion'].apply(lambda fecha: NormalFecha(fecha))
datos['Fecha_diagnostico']=datos['Fecha_diagnostico'].apply(lambda fecha: NormalFecha(fecha))
datos['Fecha_recuperado']=datos['Fecha_recuperado'].apply(lambda fecha: NormalFecha(fecha))
datos['Fecha_muerte']=datos['Fecha_muerte'].apply(lambda fecha: NormalFecha(fecha))
datos['Fecha_web']=datos['Fecha_web'].apply(lambda fecha: NormalFecha(fecha))

#Filtrar Diagnosticados
#datos_diag=pd.DataFrame(datos[np.logical_and(datos['Fecha_diagnostico']!='SIN DATO',datos['Fecha_diagnostico']!='nan')]) #2020-07-05
#datos_diag=pd.DataFrame(datos[np.logical_and(datos['Fecha_diagnostico']!='1899-12-31T00:00:00.000',datos['Fecha_diagnostico']!='nan')]) # 2020-07-06
datos_diag=pd.DataFrame(datos[datos['Fecha_diagnostico']!='NaN']) #2020-07-07

# Filtrar No Diagnosticados
#datos_nodiag=pd.DataFrame(datos[np.logical_or(datos['Fecha_diagnostico']=='SIN DATO',datos['Fecha_diagnostico']=='nan')]) #2020-07-05
#datos_nodiag=pd.DataFrame(datos[np.logical_or(datos['Fecha_diagnostico']=='1899-12-31T00:00:00.000',datos['Fecha_diagnostico']=='nan')]) # 2020-07-06
datos_nodiag=pd.DataFrame(datos[datos['Fecha_diagnostico']=='NaN']) #2020-07-07

print("\nObservaciones Diagnosticadas   :", len(datos_diag))
print("Observaciones No Diagnosticadas:  ", len(datos_nodiag))
print("Observaciones Totales          :", datos_todo)


Observaciones Diagnosticadas   : 698063
Observaciones No Diagnosticadas:   4025
Observaciones Totales          : 702088


In [13]:
datos_nodiag.sample(5)

,ID_caso,Fecha_inicio_sintoma,Fecha_notificacion,Fecha_diagnostico,Fecha_recuperado,Fecha_muerte,Fecha_web,Estado,Atencion,Tipo_recuperacion,...,DIVIPOLA,Departamento_Distrito,Codigo_departamento,Edad,Sexo,Tipo_caso,Pais_procedencia,Codigo_pais,Pertenencia_etnica,Nombre_etnia
148263,148304,2020-04-23,2020-04-26,NaN,2020-07-15,NaN,2020-07-12,Leve,Recuperado,Tiempo,...,13001,Cartagena D.T. y C.,13,27,F,En estudio,NaN,NaN,Otro,NaN
152346,152387,2020-06-16,2020-06-25,NaN,2020-07-18,NaN,2020-07-13,Leve,Recuperado,Tiempo,...,13001,Cartagena D.T. y C.,13,12,F,En estudio,NaN,NaN,Otro,NaN
236473,236514,2020-07-14,2020-07-18,NaN,2020-08-13,NaN,2020-07-25,Leve,Recuperado,Tiempo,...,17001,Caldas,17,22,M,En estudio,NaN,NaN,Otro,NaN
92863,92904,2020-05-25,2020-05-29,NaN,2020-06-29,NaN,2020-06-29,Leve,Recuperado,Tiempo,...,13001,Cartagena D.T. y C.,13,22,M,En estudio,NaN,NaN,Otro,NaN
80700,80741,2020-06-16,2020-06-24,NaN,2020-07-16,NaN,2020-06-26,Leve,Recuperado,Tiempo,...,8001,Barranquilla D.E.,8,25,M,En estudio,NaN,NaN,Negro,NaN


In [14]:
datos_diag.sample(5)


,ID_caso,Fecha_inicio_sintoma,Fecha_notificacion,Fecha_diagnostico,Fecha_recuperado,Fecha_muerte,Fecha_web,Estado,Atencion,Tipo_recuperacion,...,DIVIPOLA,Departamento_Distrito,Codigo_departamento,Edad,Sexo,Tipo_caso,Pais_procedencia,Codigo_pais,Pertenencia_etnica,Nombre_etnia
6414,6455,2020-04-27,2020-04-27,2020-04-30,2020-05-15,NaN,2020-04-30,Leve,Recuperado,PCR,...,11001,Bogotá D.C.,11,18,M,Relacionado,NaN,NaN,Otro,NaN
553020,553061,2020-08-16,2020-08-22,2020-08-24,2020-09-08,NaN,2020-08-25,Leve,Recuperado,Tiempo,...,5001,Antioquia,5,32,M,En estudio,NaN,NaN,Otro,NaN
19905,19946,2020-05-19,2020-05-19,2020-05-23,2020-06-04,NaN,2020-05-23,Leve,Recuperado,PCR,...,73268,Tolima,73,41,M,Relacionado,NaN,NaN,Negro,NaN
617966,618007,2020-08-22,2020-08-31,2020-08-30,NaN,NaN,2020-09-01,Leve,Casa,NaN,...,86001,Putumayo,86,42,F,En estudio,NaN,NaN,Otro,NaN
62338,62379,2020-06-08,2020-06-08,2020-06-19,2020-06-24,NaN,2020-06-19,Leve,Recuperado,PCR,...,11001,Bogotá D.C.,11,57,F,En estudio,NaN,NaN,Otro,NaN
